<a href="https://colab.research.google.com/github/Misha-private/Demo-repo/blob/main/Shallow_WaterModel_KleinBeta_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ===========================
# Script 1: gen_klein_ics.py
# ===========================
import os
import numpy as np

# --- Try to import helpers; if missing, define minimal fallbacks ---
try:
    from save_utils import save_ic_bundle, DRIVE_ICS
except Exception:
    # Fallback locations suitable for Colab
    DRIVE_ICS = "/content/drive/MyDrive/klein_ics"
    os.makedirs(DRIVE_ICS, exist_ok=True)

    def save_ic_bundle(bundle_npz_path, **arrays):
        os.makedirs(os.path.dirname(bundle_npz_path), exist_ok=True)
        np.savez_compressed(bundle_npz_path, **arrays)
        return bundle_npz_path

# -------- Grid & physics (Klein-β plane) --------
Lx, Ly = 2.0e7, 1.0e7
nx, ny = 256, 128
dx, dy = Lx/nx, Ly/ny
g, H = 9.81, 1000.0
beta_like = 2e-11   # used only for smooth geostrophic init (not for dynamics)
f0 = 0.0

# Klein-β coordinates: x ∈ [-π, π), y ∈ [-π/2, π/2)
x = np.linspace(-np.pi, np.pi, nx, endpoint=False)
y = np.linspace(-np.pi/2, np.pi/2, ny, endpoint=False)
X, Y = np.meshgrid(x, y)

def rh_wave(m=4, amp=100.0):
    # Simple standing RH-like mode on Klein plane (for diversity only)
    return amp * np.cos(m * X) * np.cos(2 * Y)

def modon(x0=0.0, y0=0.0, R=0.6, amp=80.0):
    # Gaussian "modon-ish" blob; Klein topology is handled by the FD model BCs
    dx_ = (X - x0)
    r2 = ((dx_ / np.pi)**2 + ((Y - y0) / (np.pi/2))**2) / (R**2)
    return amp * np.exp(-r2)

def geostrophic_from_eta(eta):
    """
    Crude geostrophic initializer:
       u = -(g/f_reg) ∂η/∂y,  v = (g/f_reg) ∂η/∂x
    with regularization near the equator.
    """
    f = beta_like * (Y)  # not used in dynamics; only to initialize smooth winds
    f_min = 1e-5
    f_reg = np.where(np.abs(f) < f_min, f_min*np.sign(f + 1e-12), f)

    # centered finite differences on the uniform grid
    dphidy = (np.pad(eta, ((1,1),(0,0)), mode='edge')[2:, :] -
              np.pad(eta, ((1,1),(0,0)), mode='edge')[:-2, :]) / (2*dy)
    dphidx = (np.pad(eta, ((0,0),(1,1)), mode='wrap')[:, 2:] -
              np.pad(eta, ((0,0),(1,1)), mode='wrap')[:, :-2]) / (2*dx)

    u = -(g / f_reg) * dphidy
    v =  (g / f_reg) * dphidx

    # Kelvin-like pinch at exact equator
    eq = (np.abs(Y) < 1e-8)
    u[eq] = -np.sqrt(g / H) * dphidy[eq]
    v[eq] = 0.0
    return u.astype(np.float32), v.astype(np.float32)

def run():
    payload = {}

    # IC #1: RH4
    eta1 = rh_wave(m=4, amp=120.0).astype(np.float32)
    u1, v1 = geostrophic_from_eta(eta1)
    payload["rh4_eta"] = eta1; payload["rh4_u"] = u1; payload["rh4_v"] = v1

    # IC #2: RH2 + modon
    eta2 = (rh_wave(m=2, amp=80.0) + modon(x0=0.0, y0=0.0, R=0.45, amp=60.0)).astype(np.float32)
    u2, v2 = geostrophic_from_eta(eta2)
    payload["mixed_RH2_modon_eta"] = eta2; payload["mixed_RH2_modon_u"] = u2; payload["mixed_RH2_modon_v"] = v2

    # IC #3: RH3 + shifted modon
    eta3 = (rh_wave(m=3, amp=90.0) + modon(x0=0.7, y0=0.3, R=0.4, amp=50.0)).astype(np.float32)
    u3, v3 = geostrophic_from_eta(eta3)
    payload["mixed_RH3_modon_eta"] = eta3; payload["mixed_RH3_modon_u"] = u3; payload["mixed_RH3_modon_v"] = v3

    # IC #4: twin modons (+/-)
    eta4 = (modon(-1.0, 0.2, 0.35, 70.0) - modon(1.0, -0.2, 0.35, 70.0)).astype(np.float32)
    u4, v4 = geostrophic_from_eta(eta4)
    payload["twin_modons_eta"] = eta4; payload["twin_modons_u"] = u4; payload["twin_modons_v"] = v4

    # IC #5: RH5 smaller
    eta5 = rh_wave(m=5, amp=60.0).astype(np.float32)
    u5, v5 = geostrophic_from_eta(eta5)
    payload["rh5_eta"] = eta5; payload["rh5_u"] = u5; payload["rh5_v"] = v5

    # IC #6: RH4 + two weaker modons
    eta6 = (rh_wave(4, 70.0) + 0.6*modon(-0.5, 0.35, 0.3, 50.0) + 0.6*modon(0.8, -0.35, 0.3, 50.0)).astype(np.float32)
    u6, v6 = geostrophic_from_eta(eta6)
    payload["mixed_RH4_2mod_eta"] = eta6; payload["mixed_RH4_2mod_u"] = u6; payload["mixed_RH4_2mod_v"] = v6

    # IC #7: single modon
    eta7 = modon(0.0, 0.0, 0.5, 90.0).astype(np.float32)
    u7, v7 = geostrophic_from_eta(eta7)
    payload["modon_eta"] = eta7; payload["modon_u"] = u7; payload["modon_v"] = v7

    # reference grids/constants
    payload["x"] = x.astype(np.float32)
    payload["y"] = y.astype(np.float32)
    payload["H"] = np.float32(H)
    payload["g"] = np.float32(g)

    out = f"{DRIVE_ICS}/klein_ic_bundle.npz"
    save_ic_bundle(out, **payload)
    print("Saved IC bundle:", out)
    print("IC keys:", [k for k in payload.keys() if k.endswith("_eta")])

if __name__ == "__main__":
    # In Colab, run this after mounting Drive:
    # from google.colab import drive; drive.mount('/content/drive')
    run()


Saved IC bundle: /content/drive/MyDrive/klein_ics/klein_ic_bundle.npz
IC keys: ['rh4_eta', 'mixed_RH2_modon_eta', 'mixed_RH3_modon_eta', 'twin_modons_eta', 'rh5_eta', 'mixed_RH4_2mod_eta', 'modon_eta']


In [3]:
# ============================================================
# Klein-β C-grid SWE (Arakawa–Lamb, vector-invariant)
# Multi-IC driver:
#  - Runs ALL ICs present in IC bundle
#  - Saves centered fields for GNN at steps [0,200,...,1200]
#  - Saves plots (eta, uc, vc) at those checkpoints
#  - Saves per-IC diagnostics (mass, energy) CSV + time-series PNG
# ============================================================
import os, numpy as np
import matplotlib.pyplot as plt

# ---------------- User paths ----------------
ROOT_OUT   = "/content/drive/MyDrive/klein_ckpt_AL_centers"
IC_BUNDLE  = "/content/drive/MyDrive/klein_ics/klein_ic_bundle.npz"

# --------------- Phys / grid ----------------
g     = 9.81
H     = 1000.0
fp    = 8.0e-5         # f(y)=fp*sin(phi)
nx    = 256
ny    = 128
Lx    = 2.0e7
Ly    = 8.0e6
dx    = Lx/nx
dy    = Ly/ny

# time stepping
dt    = 30.0
nt    = 1200
save_steps = [0, 200, 400, 600, 800, 1000, 1200]

# diffusion / stabilization
nu_u  = 1.0e4
nu_v  = 1.0e4
nu_h  = 5.0e3
nu4_u = 5.0e10
nu4_v = 5.0e10
nu4_h = 2.5e10

# ---------- Klein geometry ----------
x_c = np.linspace(0.5*dx, Lx-0.5*dx, nx)               # centers
y_c = np.linspace(0.5*dy, Ly-0.5*dy, ny)
Xc, Yc = np.meshgrid(x_c, y_c)                          # (ny,nx)

phi_c = np.pi*((Yc/Ly) - 0.5)
f_c   = fp * np.sin(phi_c)                              # (ny,nx)
y_m   = y_c.copy().astype(np.float32)                   # store for ML

# face grids
x_u = np.linspace(0.0, Lx, nx+1)        # u on (ny,nx+1)
y_v = np.linspace(0.0, Ly, ny+1)        # v on (ny+1,nx)
Xu, Yu = np.meshgrid(x_u, y_c)
Xv, Yv = np.meshgrid(x_c, y_v)
phi_u = np.pi*((Yu/Ly) - 0.5)
phi_v = np.pi*((Yv/Ly) - 0.5)
f_u   = fp * np.sin(phi_u)
f_v   = fp * np.sin(phi_v)

# ---------- C-grid helpers ----------
def avg_x(a):  # (ny,nx) -> (ny,nx+1)
    return 0.5*(np.pad(a,((0,0),(1,0)),mode='wrap') + np.pad(a,((0,0),(0,1)),mode='wrap'))
def avg_y(a):  # (ny,nx) -> (ny+1,nx)
    return 0.5*(np.pad(a,((1,0),(0,0)),mode='edge') + np.pad(a,((0,1),(0,0)),mode='edge'))

def center_from_u(u):  # (ny,nx+1)->(ny,nx)
    return 0.5*(u[:,:-1] + u[:,1:])
def center_from_v(v):  # (ny+1,nx)->(ny,nx)
    return 0.5*(v[:-1,:] + v[1:,:])

def ddx_c_to_u(phi):   # (ny,nx)->(ny,nx+1)
    L = np.pad(phi,((0,0),(1,0)),mode='wrap'); R = np.pad(phi,((0,0),(0,1)),mode='wrap')
    return (R-L)/(2*dx)
def ddy_c_to_v(phi):   # (ny,nx)->(ny+1,nx)
    T = np.pad(phi,((1,0),(0,0)),mode='edge'); B = np.pad(phi,((0,1),(0,0)),mode='edge')
    return (B-T)/(2*dy)

def ddx_u_to_c(phi_u): # (ny,nx)
    return (phi_u[:,1:] - phi_u[:,:-1])/dx
def ddy_v_to_c(phi_v): # (ny,nx)
    return (phi_v[1:,:] - phi_v[:-1,:])/dy

# ---------- Laplacians (aligned) ----------
def lap_u(u):  # (ny, nx+1)
    ue = np.pad(u, ((0,0),(1,1)), mode='wrap')
    u_xx = (ue[:, :-2] - 2*ue[:, 1:-1] + ue[:, 2:]) / dx**2
    ue2 = np.pad(u, ((1,1),(0,0)), mode='edge')
    u_yy = (ue2[:-2, :] - 2*ue2[1:-1, :] + ue2[2:, :]) / dy**2
    return u_xx + u_yy

def lap_v(v):  # (ny+1, nx)
    ve = np.pad(v, ((0,0),(1,1)), mode='wrap')
    v_xx = (ve[:, :-2] - 2*ve[:, 1:-1] + ve[:, 2:]) / dx**2
    ve2 = np.pad(v, ((1,1),(0,0)), mode='edge')
    v_yy = (ve2[:-2, :] - 2*ve2[1:-1, :] + ve2[2:, :]) / dy**2
    return v_xx + v_yy

def lap_c(h):  # (ny, nx)
    he = np.pad(h, ((0,0),(1,1)), mode='wrap')
    h_xx = (he[:, :-2] - 2*he[:, 1:-1] + he[:, 2:]) / dx**2
    he2 = np.pad(h, ((1,1),(0,0)), mode='edge')
    h_yy = (he2[:-2, :] - 2*he2[1:-1, :] + he2[2:, :]) / dy**2
    return h_xx + h_yy

def bih_u(u): return lap_u(lap_u(u))
def bih_v(v): return lap_v(lap_v(v))
def bih_c(h): return lap_c(lap_c(h))

# ---------- Klein twist BCs ----------
def twist_reflect_x(arr):
    return arr[..., ::-1]

def apply_bc(u, v, h):
    # centers (h): even under twist
    h[0, :]  = 0.5*(h[1, :] + twist_reflect_x(h[1, :]))
    h[-1, :] = 0.5*(h[-2,:] + twist_reflect_x(h[-2,:]))
    # u-faces: odd
    u[0, :]  = 0.5*(u[1, :] - twist_reflect_x(u[1, :]))
    u[-1, :] = 0.5*(u[-2,:] - twist_reflect_x(u[-2,:]))
    # v-faces: even
    v[0, :]  = 0.5*(v[1, :] + twist_reflect_x(v[1, :]))
    v[-1, :] = 0.5*(v[-2,:] + twist_reflect_x(v[-2,:]))
    return u, v, h

# ---------- Vorticity + AL RHS ----------
def compute_corner_vorticity(u, v):
    v_w = np.pad(v,((0,0),(1,0)),mode='wrap'); v_e = np.pad(v,((0,0),(0,1)),mode='wrap')
    dv_dx = (v_e - v_w)/(2*dx)
    u_s = np.pad(u,((1,0),(0,0)),mode='edge'); u_n = np.pad(u,((0,1),(0,0)),mode='edge')
    du_dy = (u_n - u_s)/(2*dy)
    return dv_dx - du_dy

def to_u_from_corners(a):  # (ny+1,nx+1)->(ny,nx+1)
    return 0.5*(a[:-1,:] + a[1:,:])
def to_v_from_corners(a):  # (ny+1,nx+1)->(ny+1,nx)
    return 0.5*(a[:,:-1] + a[:,1:])

def rhs(u, v, h):
    u, v, h = apply_bc(u, v, h)

    # mass fluxes
    h_u = avg_x(h)
    h_v = avg_y(h)

    # pressure + KE gradients
    uc = center_from_u(u); vc = center_from_v(v)
    K  = 0.5*(uc**2 + vc**2)
    Phi = g*h
    dPhidx_u = ddx_c_to_u(Phi)
    dPhidy_v = ddy_c_to_v(Phi)
    dKdx_u   = ddx_c_to_u(K)
    dKdy_v   = ddy_c_to_v(K)

    # absolute vorticity to faces
    zeta_corners = compute_corner_vorticity(u, v)
    zeta_u = to_u_from_corners(zeta_corners)
    zeta_v = to_v_from_corners(zeta_corners)
    eta_u  = zeta_u + f_u
    eta_v  = zeta_v + f_v

    # transverse velocities
    v_tu   = avg_x(center_from_v(v))
    u_tv   = avg_y(center_from_u(u))

    # momentum tendencies (vector-invariant w/ KE gradient)
    du = -(dPhidx_u + dKdx_u) + eta_u * v_tu + nu_u*lap_u(u) + nu4_u*bih_u(u)
    dv = -(dPhidy_v + dKdy_v) - eta_v * u_tv + nu_v*lap_v(v) + nu4_v*bih_v(v)

    # continuity (flux form)
    F_u = h_u * u
    F_v = h_v * v
    dFdx = ddx_u_to_c(F_u)
    dFdy = ddy_v_to_c(F_v)
    dhdt = -(dFdx + dFdy) + nu_h*lap_c(h) + nu4_h*bih_c(h)

    du, dv, dhdt = apply_bc(du, dv, dhdt)
    return du, dv, dhdt

# ---------- RK4 ----------
def rk4_step(u, v, h, dt):
    k1u,k1v,k1h = rhs(u, v, h)
    u2,v2,h2 = apply_bc(u + 0.5*dt*k1u, v + 0.5*dt*k1v, h + 0.5*dt*k1h)

    k2u,k2v,k2h = rhs(u2, v2, h2)
    u3,v3,h3 = apply_bc(u + 0.5*dt*k2u, v + 0.5*dt*k2v, h + 0.5*dt*k2h)

    k3u,k3v,k3h = rhs(u3, v3, h3)
    u4,v4,h4 = apply_bc(u + dt*k3u, v + dt*k3v, h + dt*k3h)

    k4u,k4v,k4h = rhs(u4, v4, h4)

    u_new = u + (dt/6.0)*(k1u + 2*k2u + 2*k3u + k4u)
    v_new = v + (dt/6.0)*(k1v + 2*k2v + 2*k3v + k4v)
    h_new = h + (dt/6.0)*(k1h + 2*k2h + 2*k3h + k4h)

    return apply_bc(u_new, v_new, h_new)

# ---------- Diagnostics ----------
def total_mass(h): return float(np.sum(h) * dx * dy)
def total_energy(u, v, h):
    uc = center_from_u(u); vc = center_from_v(v)
    ke = 0.5*h*(uc**2 + vc**2)
    pe = 0.5*g*(h - H)**2
    return float(np.sum((ke + pe) * dx * dy))

# ---------- IC loading (centered) ----------
def load_ic_centered(ic_key, path=IC_BUNDLE):
    d = np.load(path)
    eta = d[f"{ic_key}_eta"].astype(np.float32)
    uc0 = d[f"{ic_key}_u"].astype(np.float32)
    vc0 = d[f"{ic_key}_v"].astype(np.float32)
    return eta, uc0, vc0

def centered_to_faces(uc, vc):
    u = np.zeros((ny, nx+1), dtype=np.float32)
    u[:,1:-1] = 0.5*(uc[:, :-1] + uc[:, 1:])
    u[:,0]    = uc[:, 0]
    u[:,-1]   = uc[:, -1]
    v = np.zeros((ny+1, nx), dtype=np.float32)
    v[1:-1,:] = 0.5*(vc[:-1,:] + vc[1:,:])
    v[0,:]    = vc[0,:]
    v[-1,:]   = vc[-1,:]
    return u, v

# ---------- Save centered snapshot ----------
def save_centered_snapshot(step, ic_key, h, u, v, t):
    uc = center_from_u(u).astype(np.float32)
    vc = center_from_v(v).astype(np.float32)
    eta= (h - H).astype(np.float32)
    ic_dir = os.path.join(ROOT_OUT, ic_key)
    os.makedirs(ic_dir, exist_ok=True)
    path = os.path.join(ic_dir, f"klein_step_{step:06d}.npz")
    np.savez_compressed(
        path,
        h=h.astype(np.float32),
        eta=eta,
        uc=uc, vc=vc,
        f=f_c.astype(np.float32),
        y_m=y_m.astype(np.float32),
        H=float(H),
        dt=float(dt),
        t=float(t),
    )
    return path

# ---------- Plot helpers ----------
def quick_plot_fields(ic_key, step, eta, uc, vc):
    pdir = os.path.join(ROOT_OUT, ic_key, "plots")
    os.makedirs(pdir, exist_ok=True)
    xkm = Xc/1e3; ykm = (Yc - 0.5*Ly)/1e3

    plt.figure(figsize=(12,4))
    plt.suptitle(f"{ic_key}  step={step}  t={step*dt/3600:.2f} h")
    ax1 = plt.subplot(1,3,1)
    im1 = ax1.pcolormesh(xkm, ykm, eta, shading="auto")
    plt.colorbar(im1, ax=ax1, label="η (m)")
    ax1.set_title("Height anomaly")
    ax1.set_xlabel("x (km)"); ax1.set_ylabel("y (km)")

    ax2 = plt.subplot(1,3,2)
    im2 = ax2.pcolormesh(xkm, ykm, uc, shading="auto")
    plt.colorbar(im2, ax=ax2, label="u_c (m/s)")
    ax2.set_title("u at centers")

    ax3 = plt.subplot(1,3,3)
    im3 = ax3.pcolormesh(xkm, ykm, vc, shading="auto")
    plt.colorbar(im3, ax=ax3, label="v_c (m/s)")
    ax3.set_title("v at centers")
    plt.tight_layout()
    out = os.path.join(pdir, f"fields_step_{step:06d}.png")
    plt.savefig(out, dpi=120)
    plt.close()

def plot_me_history(ic_key, steps, mass, energy):
    pdir = os.path.join(ROOT_OUT, ic_key, "plots")
    os.makedirs(pdir, exist_ok=True)
    tdays = np.array(steps)*dt/86400.0
    plt.figure(figsize=(8,4))
    ax = plt.gca()
    ax.plot(tdays, (mass - mass[0])/mass[0], label="ΔM/M0")
    ax.plot(tdays, (energy - energy[0])/energy[0], label="ΔE/E0")
    ax.set_xlabel("time (days)"); ax.set_ylabel("relative change")
    ax.grid(True, alpha=0.3); ax.legend()
    out = os.path.join(pdir, "mass_energy_timeseries.png")
    plt.savefig(out, dpi=120)
    plt.close()

# ---------- Run one IC ----------
def run_one_ic(ic_key):
    print(f"\n=== IC: {ic_key} | nx={nx}, ny={ny}, dt={dt:.1f}s, nt={nt} ===")
    eta0, uc0, vc0 = load_ic_centered(ic_key, IC_BUNDLE)
    assert eta0.shape==(ny,nx) and uc0.shape==(ny,nx) and vc0.shape==(ny,nx), "IC bundle shapes must match grid."

    h = (H + eta0).astype(np.float32)
    u, v = centered_to_faces(uc0, vc0)
    u, v, h = apply_bc(u, v, h)

    M0 = total_mass(h); E0 = total_energy(u, v, h)
    print(f"Initial mass={M0:.6e}, energy={E0:.6e}")

    # diagnostics arrays
    me_steps, me_mass, me_energy = [], [], []

    # save step 0
    if 0 in save_steps:
        p = save_centered_snapshot(0, ic_key, h, u, v, t=0.0)
        uc = center_from_u(u); vc = center_from_v(v); eta = h - H
        quick_plot_fields(ic_key, 0, eta, uc, vc)
        print(f"[save] {ic_key} step=0 -> {p}")

    t = 0.0
    for n in range(1, nt+1):
        u, v, h = rk4_step(u, v, h, dt)
        t += dt

        # mild safety
        if np.any(h < 1.0): h[:] = np.maximum(h, 1.0)

        if (n in save_steps):
            p = save_centered_snapshot(n, ic_key, h, u, v, t)
            uc = center_from_u(u); vc = center_from_v(v); eta = h - H
            quick_plot_fields(ic_key, n, eta, uc, vc)
            print(f"[save] {ic_key} step={n} -> {p}")

        if (n % 50) == 0 or n == 1:
            M = total_mass(h); E = total_energy(u, v, h)
            me_steps.append(n); me_mass.append(M); me_energy.append(E)
            dM = (M - M0)/M0; dE = (E - E0)/E0
            uc = center_from_u(u); vc = center_from_v(v)
            umax = float(np.max(np.sqrt(uc*uc + vc*vc)))
            print(f"[{n:5d}] dM/M0={dM:+.3e}  dE/E0={dE:+.3e}  umax={umax:6.2f}")

    # save diagnostics
    ic_dir = os.path.join(ROOT_OUT, ic_key)
    os.makedirs(ic_dir, exist_ok=True)
    diag_csv = os.path.join(ic_dir, "diagnostics_me.csv")
    if me_steps:
        arr = np.stack([np.array(me_steps), np.array(me_mass), np.array(me_energy)], axis=1)
        header = "step,mass,energy"
        np.savetxt(diag_csv, arr, delimiter=",", header=header, comments="")
        plot_me_history(ic_key, np.array(me_steps), np.array(me_mass), np.array(me_energy))
        print(f"[diag] saved {diag_csv}")

    print(f"Done: {ic_key}")

# ---------- Run ALL ICs found in bundle ----------
def list_ic_keys(bundle_path):
    d = np.load(bundle_path)
    names = [k for k in d.files if k.endswith("_eta")]
    keys = sorted(k[:-4] for k in names)  # strip "_eta"
    return keys

def main():
    os.makedirs(ROOT_OUT, exist_ok=True)
    keys = list_ic_keys(IC_BUNDLE)
    print("ICs found in bundle:", keys)
    for ic in keys:
        run_one_ic(ic)

if __name__ == "__main__":
    # from google.colab import drive; drive.mount('/content/drive')  # if needed
    main()


ICs found in bundle: ['mixed_RH2_modon', 'mixed_RH3_modon', 'mixed_RH4_2mod', 'modon', 'rh4', 'rh5', 'twin_modons']

=== IC: mixed_RH2_modon | nx=256, ny=128, dt=30.0s, nt=1200 ===
Initial mass=1.615218e+17, energy=1.093415e+20
[save] mixed_RH2_modon step=0 -> /content/drive/MyDrive/klein_ckpt_AL_centers/mixed_RH2_modon/klein_step_000000.npz
[    1] dM/M0=-1.148e-06  dE/E0=+2.670e-05  umax= 68.14
[   50] dM/M0=-1.731e-03  dE/E0=+1.995e-02  umax= 68.14
[  100] dM/M0=-5.290e-03  dE/E0=+5.245e-02  umax= 68.18
[  150] dM/M0=-9.829e-03  dE/E0=+8.964e-02  umax= 77.11
[save] mixed_RH2_modon step=200 -> /content/drive/MyDrive/klein_ckpt_AL_centers/mixed_RH2_modon/klein_step_000200.npz
[  200] dM/M0=-1.502e-02  dE/E0=+1.309e-01  umax= 87.47
[  250] dM/M0=-2.057e-02  dE/E0=+1.769e-01  umax= 96.52
[  300] dM/M0=-2.630e-02  dE/E0=+2.197e-01  umax=103.83
[  350] dM/M0=-3.209e-02  dE/E0=+2.718e-01  umax=109.30
[save] mixed_RH2_modon step=400 -> /content/drive/MyDrive/klein_ckpt_AL_centers/mixed_RH2_